In [22]:
import discogs_client
from dotenv import load_dotenv
import requests
import os
import time
from lastfm_get_album import get_album_info
load_dotenv() 
DISCOGS_CLIENTNAME= os.getenv("DISCOGS_CLIENTNAME")
DISCOGS_TOKEN=os.getenv("DISCOGS_TOKEN")

In [23]:
d = discogs_client.Client(user_agent=DISCOGS_CLIENTNAME
                          ,user_token=DISCOGS_TOKEN)

me=d.identity()
print(me)

<User 28001809 'cornelas90'>


In [ ]:
for item in me.collection_folders[0].releases:
    release = item.release 
    raw = release.data
    print(raw)

    ### Artist Info ###
    for artist in release.artists:
        raw = artist.data
        results = d.search(artist.name, type='artist')
        print(raw)

        if results:
            artist = results[0]
            print(f"Artist Name: {artist.name}")
            print(f"Artist ID: {artist.id}")
        #if artist.profile:
        #    print(f"Profile: {artist.profile}")
        #if artist.members:
        #    print("Members:")
        #    for member in artist.members:
        #        print(f"  - {member.name}")
        if artist.aliases:
            print("Aliases:")
            for alias in artist.aliases:
                print(f"  - {alias.name}")
        #if artist.images:
        #    print(f"Image URL: {artist.images[0]['uri']}")
    time.sleep(1)
    for track in release.tracklist:
        #raw = track.data # JSON
        print(raw)
    #for track in release.tracklist:
        print(f"{track.position} - {track.title} ({track.duration})")
        #print(track.extraartists)
    

{'id': 1691593, 'master_id': 178929, 'master_url': 'https://api.discogs.com/masters/178929', 'resource_url': 'https://api.discogs.com/releases/1691593', 'thumb': 'https://i.discogs.com/WeZEOTjwF-mJA9WUEztl1B-fGt_U9Sj_ifp7sjgabfg/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTE2OTE1/OTMtMTMwNzMwMjM5/My5qcGVn.jpeg', 'cover_image': 'https://i.discogs.com/mUV-bGEyxXWnnG1IKhig4hRXO4F570RC88MujEyjIiQ/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTE2OTE1/OTMtMTMwNzMwMjM5/My5qcGVn.jpeg', 'title': 'Flat As A Pancake', 'year': 1975, 'formats': [{'name': 'Vinyl', 'qty': '1', 'text': 'Terre Haute Pressing', 'descriptions': ['LP', 'Album']}], 'labels': [{'name': 'A&M Records', 'catno': 'SP-4537', 'entity_type': '1', 'entity_type_name': 'Label', 'id': 904, 'resource_url': 'https://api.discogs.com/labels/904'}], 'artists': [{'name': 'Head East', 'anv': '', 'join': '', 'role': '', 'tracks': '', 'id': 266027, 'resource_url': 'https://api.discogs.com/a